<a href="https://colab.research.google.com/github/Gus-oss/MachineLearning/blob/main/MachineLearningTarea2_CargarDeBaseDeDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Machine Learning: Análisis de los datos sobre influenza, COVID-19 y otros virus respiratorios.

## Librerias y creación de carpeta de imagenes.

In [ ]:
#Importamos las librerias que utilizaremos a lo largo del documento
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
import os #Se utilizo para guardar las imagenes (carpeta) y para guardar las imagenes en la carpeta
import plotly.graph_objects as go #esta libreria es para hacer graficos dinamicos, como por ejemplo
#que al pasar el cursor se viera el valor real

# Librerias para Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Carpeta única para guardar las imágenes
IMAGES_PATH = "Galeria"  # Nombre de la carpeta (puedes cambiarlo)
os.makedirs(IMAGES_PATH, exist_ok=True) # Crea la carpeta si no existe

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id, "in", IMAGES_PATH) #Mensaje mas informativo
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Obtencion de datos.

Los datos se actualizan cada cierto periodo de tiempo. Para esto, se automatizo la optención de los datos para que al momento de ejecutar el programa se estudien los datos mas recientes posibles publicados por la Direccion de Epidemiología de México.

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL de la página de datos abiertos
url = 'https://www.gob.mx/salud/documentos/datos-abiertos-152127'

# Obtener el HTML de la página
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.content, 'html.parser')

# Imprimir todos los enlaces en la página para ver su estructura
print("Enlaces encontrados en la página:")
for a in soup.find_all('a', href=True):
    print(a['href'])

Enlaces encontrados en la página:
/
/tramites
/gobierno
/salud/en
https://www.gob.mx/busqueda?utf8=✓
?tab=
/salud/es/archivo/prensa
/salud/es/#390
/salud/es/archivo/documentos
/salud/es/archivo/acciones_y_programas
/salud/es/archivo/multimedia
http://portal.salud.gob.mx/sites/transparencia/transparencia/proteccion_datos/proteccion_1_1.html
http://portal.salud.gob.mx/sites/transparencia/transparencia/transparencia/transparencia_1.html
http://www.gob.mx
/salud
/salud/articulos/destaca-el-inper-en-avances-tecnologicos-en-obstetricia-y-atencion-del-embarazo
/salud/articulos/un-tratamiento-adecuado-para-el-sindrome-de-turner-permite-el-buen-desarrollo-de-las-ninas-inp
/salud/articulos/realizan-sesion-cultural-en-el-marco-del-178-aniversario-del-hospital-juarez-de-mexico
/salud/articulos/revision-oftalmologica-clave-en-la-deteccion-de-la-enfermedad-de-coats
/salud/articulos/programa-micaela-brinda-atencion-integral-a-mujeres-con-cancer-cervicouterino-incan
https://www.gob.mx/salud/documentos

In [ ]:
import requests
import zipfile
import io

# URL del Diccionario de datos
diccionario_url = 'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_abiertos.zip'

# Descargar y abrir el archivo ZIP del Diccionario de datos
diccionario_zip_response = requests.get(diccionario_url)
diccionario_zip_response.raise_for_status()

with zipfile.ZipFile(io.BytesIO(diccionario_zip_response.content)) as diccionario_zip:
    # Listar los nombres de los archivos en el ZIP
    print("Archivos contenidos en el ZIP del Diccionario de datos:")
    for file_name in diccionario_zip.namelist():
        print(file_name)

Archivos contenidos en el ZIP del Diccionario de datos:
240708 Catalogos.xlsx
240708 Descriptores_.xlsx
Actualizaciones en la presentación de información referente a casos de COVID.pdf


In [ ]:
import requests
import zipfile
import io
import pandas as pd

# URLs directas para los archivos ZIP
covid_data_url = 'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_influenza_covid19.zip'
diccionario_url = 'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_abiertos.zip'

def descargar_datos_covid():
    # Descargar y procesar el archivo ZIP de datos COVID-19
    print("Descargando archivo ZIP de datos COVID-19 desde:", covid_data_url)
    covid_zip_response = requests.get(covid_data_url)
    covid_zip_response.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(covid_zip_response.content)) as covid_zip:
        for file_name in covid_zip.namelist():
            if file_name.endswith('.csv'):
                print(f"Extrayendo {file_name}...")
                with covid_zip.open(file_name) as csv_file:
                    df_covid = pd.read_csv(csv_file)
                    print("Datos COVID-19 cargados correctamente en el DataFrame.")
                    break  # Salir después de encontrar el archivo CSV

    # Descargar y procesar el archivo ZIP del Diccionario de datos
    print("Descargando archivo ZIP del Diccionario de datos desde:", diccionario_url)
    diccionario_zip_response = requests.get(diccionario_url)
    diccionario_zip_response.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(diccionario_zip_response.content)) as diccionario_zip:
        # Cargar los archivos Excel
        excel_file_catalogos = pd.read_excel(diccionario_zip.open('240708 Catalogos.xlsx'))
        df_descriptores = pd.read_excel(diccionario_zip.open('240708 Descriptores_.xlsx'))
        print("Archivos del Diccionario de datos cargados correctamente en DataFrames.")

        # Cargar hojas específicas en DataFrames separados
        df_enti = pd.read_excel(diccionario_zip.open('240708 Catalogos.xlsx'), sheet_name='Catálogo de ENTIDADES')
        df_muni = pd.read_excel(diccionario_zip.open('240708 Catalogos.xlsx'), sheet_name='Catálogo MUNICIPIOS')
        df_sector = pd.read_excel(diccionario_zip.open('240708 Catalogos.xlsx'), sheet_name='Catálogo SECTOR')
        df_pcr = pd.read_excel(diccionario_zip.open('240708 Catalogos.xlsx'), sheet_name='Catálogo RESULTADO_PCR')

    return df_covid, excel_file_catalogos, df_descriptores, df_enti, df_muni, df_sector, df_pcr

# Ejecutar la función y cargar los datos en los DataFrames
df_covid, excel_file_catalogos, df_descriptores, df_enti, df_muni, df_sector, df_pcr = descargar_datos_covid()

Descargando archivo ZIP de datos COVID-19 desde: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_influenza_covid19.zip
Extrayendo COVID19MEXICO.csv...
Datos COVID-19 cargados correctamente en el DataFrame.
Descargando archivo ZIP del Diccionario de datos desde: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_abiertos.zip
Archivos del Diccionario de datos cargados correctamente en DataFrames.


## Limpieza de la base de datos.

In [ ]:
df_covid #visualizamos la base de datos

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,RESULTADO_PCR,RESULTADO_PCR_COINFECCION,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL_COVID,CLASIFICACION_FINAL_FLU,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2025-09-09,gd8311d,1,8,9,1,9,9,12,1,...,997,997,2,97,6,6,99,México,97,97
1,2025-09-09,g98e198,1,15,21,1,21,21,114,2,...,3,998,2,97,7,3,99,México,97,2
2,2025-09-09,g5075fe,1,6,9,1,29,9,5,2,...,5,5,2,97,7,7,99,México,97,2
3,2025-09-09,ga8a474,1,4,20,2,20,20,413,1,...,997,997,2,97,6,6,99,México,97,97
4,2025-09-09,gd25b0d,1,8,9,2,9,9,7,1,...,997,997,2,97,6,6,99,México,97,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110098,2025-09-09,g8fec22,1,12,12,1,12,12,44,2,...,999,999,2,97,6,6,99,México,97,2
110099,2025-09-09,g86c05b,1,12,14,2,14,14,37,1,...,997,997,2,97,6,6,99,México,97,97
110100,2025-09-09,g89cff3,1,3,21,1,21,21,114,1,...,997,997,2,97,6,6,99,México,97,97
110101,2025-09-09,gebab51,1,12,4,2,4,4,2,1,...,997,997,2,97,6,6,99,México,97,97
